In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from lvmdrp.scripts import preproc


OBS_NAME = "dummy-desi-obs.fits.gz"

import re


type2fmt = {
    "bool": "L",
    "binary": "L",
    "logical": "L",
    "byte": "X",
    "char": "A",
    "int16": "I",
    "int32": "J",
    "int64": "K",
    "float32": "E",
    "float64": "D"
}
type2dmy = dict(zip(type2fmt.keys(),[True, True, True, 0, "x", -9999, -9999, -9999, -9999.9, -9999.9]))

pattern = re.compile(r"\[(\d+)\]")

In [2]:
import pandas as pd


raw_data_url = "https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_DATA/NIGHT/EXPID/desi-EXPID.html"

desimodel_raw_tables = []
for table in pd.read_html(raw_data_url):
    clean_table = table.fillna(-9999)
    clean_table = clean_table.applymap(lambda s: s.replace("’","").replace("‘","").replace("…","") if type(s)==str else s)
    clean_table = clean_table.applymap(lambda s: pd.to_numeric(s, errors="ignore", downcast="integer"))
    desimodel_raw_tables.append(clean_table)

In [3]:
desimodel_raw_structure = desimodel_raw_tables[1]
desimodel_raw_structure

,Number,EXTNAME,Type,Contents
0,HDU0,SPS,IMAGE,Blank except for header keywords
1,HDU1,Z0,IMAGE,Raw data from the Z0 spectrograph
2,HDU2 – HDU30,various,IMAGE,Raw data similar to Z0 spectrograph.
3,HDU31,SPECTCONS,BINTABLE,Telemetry data


In [4]:
desimodel_raw_sps = desimodel_raw_tables[2]
desimodel_raw_sps

,KEY,Example Value,Type,Comment
0,OBSERVAT,KPNO,str,Observatory name
1,OBS-LAT,31.96403,str,[deg] Observatory latitude TYPE: Should be float.
2,OBS-LONG,-111.59989,str,[deg] Observatory east longitude TYPE: Should ...
3,OBS-ELEV,-9999,float,[m] Observatory elevation
4,TELESCOP,KPNO 4.0-m telescope,str,Telescope name
...,...,...,...,...
155,TELDEC,19.88,float,Telescope pointing dec [degrees] MISSING
156,AIRMASS,1,float,Airmass at middle of exposure MISSING
157,TILEID,4,int,DESI tile ID MISSING
158,DEPNAM00,python,str,MISSING


In [5]:
desimodel_raw_cam = desimodel_raw_tables[3]
desimodel_raw_cam

,KEY,Example Value,Type,Comment
0,NAXIS1,8,int,width of table in bytes
1,NAXIS2,4194,int,number of rows in table
2,BZERO,32768,int,offset data range to that of unsigned short
3,BSCALE,1,int,default scaling factor
4,INSTRUME,DESI,str,Instrument name
...,...,...,...,...
126,RDNOISEA,3,float,"Expected readnoise from ICS, not measured from..."
127,RDNOISEB,3,float,MISSING
128,RDNOISEC,3,float,MISSING
129,RDNOISED,3,float,MISSING


In [6]:
desimodel_raw_con = desimodel_raw_tables[4]
desimodel_raw_con

,KEY,Example Value,Type,Comment
0,NAXIS1,248,int,width of table in bytes
1,NAXIS2,1,int,number of rows in table
2,INSTRUME,DESI,str,Instrument name
3,PROCTYPE,RAW,str,Data processing level
4,PRODTYPE,image,str,Data product type
5,PROGRAM,spec tests,str,Program name
6,EXPID,1209,int,Exposure number
7,EXPFRAME,0,int,Frame number
8,FILENAME,/exposures/desi/specs/20190306/00001209/sp0-00...,str,Name UNEXPECTED: Why is this here?
9,FLAVOR,dark,str,Observation type


In [7]:
desimodel_raw_tbl = desimodel_raw_tables[5]
desimodel_raw_tbl

,Name,Type,Units,Description
0,unit,int32,-9999.0,UNEXPECTED: Why is this not all-caps?
1,DATE-OBS,char[26],-9999.0,"WARNING: Remove hyphen! Also, isnt this the sa..."
2,TIME-OBS,char[15],-9999.0,"WARNING: Remove hyphen! Also, isnt this the sa..."
3,MJD-OBS,float32,-9999.0,"WARNING: Remove hyphen! Also, isnt this the sa..."
4,ST,char[12],-9999.0,label for field 5
5,OBSID,char[19],-9999.0,label for field 6
6,STATUS,char[5],-9999.0,label for field 7
7,HARTL,char[4],-9999.0,label for field 8
8,HARTLP,char[3],-9999.0,label for field 9
9,HARTR,char[4],-9999.0,label for field 10


In [8]:
# generate dummy observations

import itertools as it

dummy = fits.HDUList()

hdu = fits.PrimaryHDU(header=fits.Header(cards=map(tuple, desimodel_raw_sps.drop(columns="Type").values.tolist())))
hdu.name = "SPS"

dummy.append(hdu)

for cam, i in it.product("Z", range(1)):
    
    hdu = fits.PrimaryHDU(
        header=fits.Header(map(tuple, desimodel_raw_cam.drop(columns="Type").values.tolist())),
        data=np.zeros((2048,2048))
    )
    hdu.name = f"{cam}{i}"
    
    dummy.append(hdu)

header = fits.Header(cards=map(tuple, desimodel_raw_con.drop(columns="Type").values.tolist()))
columns = []
for i, row in desimodel_raw_tbl.iterrows():

    shape = re.findall(pattern, row.Type)
    ctype = re.sub(pattern, "", row.Type)
    if len(shape) != 0:
        fmt = "%s%s"%(shape[0],type2fmt[ctype])
        arr = int(shape[0])*[type2dmy[ctype]] if ctype!="char" else [int(shape[0])*type2dmy[ctype]]
    else:
        fmt = type2fmt[ctype]
        arr = [type2dmy[ctype]]

    columns.append(fits.Column(name=row.Name, format=fmt, array=arr, unit=row.Units if row.Units!=-9999 else None))

hdu = fits.BinTableHDU.from_columns(header=header, columns=columns)
hdu.name = "SPECTCONS"
dummy.append(hdu)

dummy.writeto(os.path.join("_dummy_data/",OBS_NAME), overwrite=True)

dummy.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  SPS           1 PrimaryHDU     165   ()      
  1  Z0            1 ImageHDU       135   (2048, 2048)   float64   
  2  SPECTCONS     1 BinTableHDU    102   1R x 35C   ['J', '26A', '15A', 'E', '12A', '19A', '5A', '4A', '3A', '4A', '3A', '5A', '4A', '8A', '2A', '6A', '8A', '2A', '8A', 'E', 'E', 'E', 'E', 'E', 'E', '3A', '3A', '3A', '3A', '7A', '7A', '7A', '3A', '3A', '3A']   
